In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), '..'))
	print(os.getcwd())
except:
	pass


 #Title: "XO"
 #Author: "Merghadi Abdelaziz"

 # Import Libraries

In [0]:
# Essentials
import numpy as np 
import pandas as pd 
import tables
from functools import partial

# For Ploting
import matplotlib.pyplot as plt
from cycler import cycler
import matplotlib as mpl
import matplotlib.gridspec as gridspec
from matplotlib.animation import FuncAnimation
import seaborn as sns
from IPython.display import HTML
%matplotlib inline
from rasterio import rio
from rasterio.plot import show

# For Scipy
from scipy import interp
from scipy.stats import wilcoxon, mannwhitneyu, friedmanchisquare, ttest_ind

# For Sci-Kit Learn
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from lightgbm import LGBMClassifier
# from catboost import CatBoostClassifier

from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier

from sklearn.model_selection import cross_validate, cross_val_score,cross_val_predict, KFold, StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, cohen_kappa_score, log_loss, make_scorer, auc, roc_curve,brier_score_loss
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
import random

# For HyperOpt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, rand, space_eval
from hyperopt.pyll import scope as hp_scope
from hyperopt.pyll.stochastic import sample as ho_sample

# OS Specific
from pathlib import Path
import time

# Geo-Data
import geopandas as gpd 
import rasterio as rio
from osgeo import gdal,ogr

#For Markdown
from tabulate import tabulate


In [0]:
# Init. Project Folder Path 
projhome = Path.cwd()

# Number of CPUs to use in Optimization
cpus = 2

# Set the RandomState of the seed:
seed = 101
rng = np.random.RandomState(seed)
np.random.seed(seed)         
print ("Random number with seed 30")
random.seed(seed)


In [0]:
# Load Input Data
def load_data(data):
    try:
        df = pd.read_csv(data)
        return df
    except IOError:
        print("IOError: The file is not available...Check if it exist!")


# Dataset Path
data_path = "./Data/Input_Data_Array.csv"
input_df = load_data(data_path)
input_df = input_df.iloc[:,1:]


In [0]:
# Compute the correlation matrix
input_cor = input_df.drop("Landslides", axis=1).corr()

# Compute info-Gain
gain_df = {"variable":input_df.drop("Landslides", axis=1).columns,
                        "gain":mutual_info_classif(input_df.drop("Landslides", axis=1), input_df["Landslides"], discrete_features=True)}
gain_df = pd.DataFrame(data=gain_df)
with open("./Output/Tables/gain_df.md",'w') as tabl:
    print(tabulate(gain_df, headers=['variable', 'gain'],tablefmt="grid"),file=tabl)

plt.style.use("seaborn-paper")
fig, ax = plt.subplots(figsize=(5,5))
gain_df.plot.barh(x="variable",y="gain",ax=ax,legend=False)
ax.xaxis.grid(False)
ax.yaxis.grid(False)
fig.tight_layout()
plt.savefig('./Output/Figures/gain_plot.jpg',quality=100, dpi=300,bbox_inchs="tight")

# Compute the VIF
vifs = pd.DataFrame(data={"vif":np.linalg.inv(input_cor.values).diagonal(),"variable":input_cor.index})
# vifs = pd.Series(np.linalg.inv(input_cor.values).diagonal(), index=input_cor.index)
# vifs.astype("float")
with open("./Output/Tables/vif_df.md",'w') as tabl:
    print(tabulate(vifs, headers=['variable', 'vif'],tablefmt="grid"),file=tabl)

plt.style.use("seaborn-paper")
fig, ax = plt.subplots(figsize=(5,5))
vifs.plot.barh(x="variable",y="vif",ax=ax,legend=False)
ax.xaxis.grid(False)
ax.yaxis.grid(False)
fig.tight_layout()
plt.savefig('./Output/Figures/vif_plot.jpg',quality=100, dpi=300,bbox_inchs="tight")


 # Init. the necessary functions

In [0]:
# Init. the splitting function:
def cv_splitting(features, labels, cv, inner_cv=None):
    '''
    0 Refer to Features
    1 Refer to Labels
    '''
    X_train_out = []
    Y_train_out = []
    X_test_out = []
    Y_test_out = []
    print("Splitting the Input data according to The resampling...")
    for i, (train_idx, test_idx) in enumerate(cv.split(features, labels)):
        print("Fold number %d...." % (i), end=" ")
        X_train_out.append(features[train_idx])
        Y_train_out.append(labels[train_idx])
        X_test_out.append(features[test_idx])
        Y_test_out.append(labels[test_idx])
        print("done.")
    return X_train_out,Y_train_out,X_test_out,Y_test_out



In [0]:
# Setting up the objective function
count = 0
def objective_func(params,xdata, clf, ydata, cv_resample,metric="roc_auc"):
    global count
    count += 1
    lrn_clf = clf
    lrn_clf = lrn_clf.set_params(**params)
    obj_scores = []
    for i, (X, Y) in enumerate(zip(xdata, ydata)):
        cv_score = cross_val_score(lrn_clf, X=X, y=Y, scoring=metric, cv=cv_resample,error_score='raise').mean()
        if cv_score is not None:
            obj_scores.append(cv_score)

    loss = -np.mean(obj_scores)
    print('''\n#############################\n Results ƪ(˘⌣˘)ʃ \n#############################\n\n\
    Iterations: %s \n\
    Loss: %.5f \n\
    using: %s \n#############################\n''' % (count, loss, params))
    return {'loss': loss, 'status': STATUS_OK}


In [0]:
# Output the optimization trails function
def optimization_trails(trails,search_space):

    best_hp = dict([(k,np.NaN) if not v else (k,v[0]) for k,v in trails.best_trial["misc"]["vals"].items()])
    best_hp = space_eval(search_space,best_hp)
    best_hp["tid"] = trails.best_trial["misc"]["tid"]
    best_hp["loss"] = trails.best_trial["result"]["loss"]
    best_hp["status"] = trails.best_trial["result"]["status"]
    best_hp = pd.DataFrame.from_dict([best_hp])

    optimization_df = []
    for idx, value in enumerate(trails.trials):
        optimization_dic = dict([(k,np.NaN) if not v else (k,v[0]) for k,v in value["misc"]["vals"].items()])
        optimization_dic = space_eval(search_space,optimization_dic)
        optimization_dic["tid"] = value["misc"]["tid"]
        optimization_dic["loss"] = value["result"]["loss"]
        optimization_dic["status"] = value["result"]["status"]
        optimization_df.append(optimization_dic)

    optimization_df = pd.DataFrame(optimization_df)  
    return optimization_df, best_hp



In [0]:
# Plotting optimization history function
def optimization_history(trails):
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.scatter(range(1, len(trails) + 1),
                [-x['result']['loss'] for x in trails],
                edgecolor='black',c="C1", linewidth=1,
                s=80, zorder=2, label="Iteration")

    ax.set_xlabel('Iteration', fontsize=12)
    # ax.set_xticks(range(1, len(trails) + 1,1))
    ax.set_ylabel('ROC-AUC', fontsize=12)
    ax.set_title('Optimization history', fontsize=14)
    ax.grid("on", linestyle='--', linewidth=1, alpha=0.3)
    return fig, ax


In [0]:
# Train and evaluate the final model
def tunned_model_evaluation(clf,opt_hp,x,y,cv,scores):
    print(f"Train & and validate using the Following Parameters: \n {opt_hp}")  
    lrn_clf = clf
    lrn_clf = lrn_clf.set_params(**opt_hp)

    # generate scores metrics
    final = cross_validate(lrn_clf, X=x, y=y, cv=cv,scoring=scores, return_train_score=True)
    # final_prob = cross_val_predict(lrn_clf, X=x, y=y, cv=cv,method="predict_proba")

    # Formating the exported scores
    scores_raw = pd.DataFrame(data=final)
    scores = pd.DataFrame({"mean":np.mean(scores_raw), "std":np.std(scores_raw)})
    return scores, scores_raw


 ## Significance test and Pair-wise analysis

In [0]:
# Wilcoxon Pair-wise analysis
def sig_stat(data_list,data_names):

    pval = []
    statval = []
    pair = []

    for idx in range(0,len(data_list),1):
        if idx < len(data_list):
            i = idx+1
            while i < len(data_list):
                t, p = ttest_ind(data_list[idx], data_list[i], equal_var = False)
                # t, p = mannwhitneyu(data_list[idx].values, data_list[i].values)
                statval.append(round(t,3))
                pval.append(round(p,4))
                pair.append(f"{data_names[idx]} vs. {data_names[i]}")
                # print(f"{idx}     {i}")
                print(f"{data_names[idx]} vs. {data_names[i]}:\nT: {t} \t p: {p}")
                i += 1
            else:
                continue
        else:
            break
    return pd.DataFrame({"Pairwise":pair,"p.value":pval,"t.value":statval})


## Classification and ROC analysis

In [0]:
# Run classifier with cross-validation and plot ROC curves
# Compute ROC curve and area the curve function
def generate_roc_data(clf,train_x,train_y,test_x,test_y):
    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)
    for x,y,xt,yt in zip(train_x,train_y,test_x,test_y):
        probas_ = clf.fit(x, y).predict_proba(xt)
        # Compute ROC curve and area the curve
        fpr, tpr, _ = roc_curve(yt, probas_[:, 1])
        tprs.append(interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        mean_tpr = np.mean(tprs, axis=0)
        mean_tpr[-1] = 1.0
        mean_auc = auc(mean_fpr, mean_tpr)
        std_auc = np.std(aucs)
        std_tpr = np.std(tprs, axis=0)
        tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
        tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    return (fpr, tpr, roc_auc), (mean_fpr, mean_tpr, mean_auc, std_auc,tprs_upper,tprs_lower,std_tpr)
# Plot all roc plots function
def plot_roc_curves(roc_data,roc_data_names):
    mpl.style.use("seaborn-paper")
    # custom_cycler = cycler('linestyle',['-','--',':','-.'])
    custom_cycler = (cycler(color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
                                '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
                                '#bcbd22', '#17becf']) + 
                                cycler(linestyle=['-','--',':','-.','-','--',':','-.','-','--'])
                                )
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.plot([0, 1], [0, 1], linestyle=':', lw=0.8, alpha=.8,
                color='k',
                # label='Luck'
                )
    ax.set_prop_cycle(custom_cycler)
    # return (fpr, tpr, roc_auc), (mean_fpr, mean_tpr, mean_auc, std_auc,tprs_upper,tprs_lower,std_tpr)
    for idx, value in enumerate(roc_data):
        ax.plot(value[0], value[1], \
                    # color='C1',
                    label=f"{roc_data_names[idx]} AUC-ROC = {value[2]:0.3f} +/- {value[3]:0.3f}",
                    lw=1.4)
                    # lw=1.4, alpha=.8)
        # ax.fill_between(value[0], value[5], value[4],
        #             # label=f"+/- 1 std. dev.",
        #             # color='grey',
        #             alpha=.1
        #             )

    ax.set_xlim([-0.05, 1.05])
    ax.set_ylim([-0.05, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    # ax.set_title('Receiver operating characteristic curve')
    ax.legend(loc="lower right")
    return fig, ax



 Calibration analysis

In [0]:
# Probability Calibration curves function
def plot_calibration_curves(clf,x,y,test_size,cv, seed, clf_name):
    from sklearn.metrics import brier_score_loss
    custom_cycler = (cycler(color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
                                '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
                                '#bcbd22', '#17becf']) + 
                                cycler(marker=['o', 'X', 's', '8', '>', '*',"h","v","p","d"]) +
                                cycler(linestyle=['-','--',':','-.','-','--',':','-.','-','--'])
                                )
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=test_size,random_state=101)
    
    fig,ax1 = plt.subplots(figsize=(7, 7))
    # fig,(ax1,ax2) = plt.subplots(2,1,figsize=(8, 8))
    ax1.set_prop_cycle(custom_cycler)
    # ax2.set_prop_cycle(custom_cycler)
    ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")

    for lrn, lrn_name in zip(clf, clf_name):
        """Plot calibration curve for clf w/o and with calibration. """
        # Calibrated with isotonic calibration
        isotonic = CalibratedClassifierCV(lrn, cv=cv, method='isotonic')
        # Calibrated with sigmoid calibration
        sigmoid = CalibratedClassifierCV(lrn, cv=cv, method='sigmoid')
        # Logistic regression with no calibration as baseline
        # lr = LogisticRegression(C=1., solver='lbfgs')
        
        def generate_calibration_data(model,X_train,y_train,X_test,y_test):
            model.fit(X_train, y_train)
            # y_pred = lrn_clf.predict(X_test)
            if hasattr(model, "predict_proba"):
                prob_pos = model.predict_proba(X_test)[:, 1]
            else:  # use decision function
                prob_pos = model.decision_function(X_test)
                prob_pos = (prob_pos - prob_pos.min()) / (prob_pos.max() - prob_pos.min())
            
            fraction_of_positives, mean_predicted_value = calibration_curve(y_test, prob_pos, n_bins=10)       
            lrn_score = brier_score_loss(y_test, prob_pos, pos_label=y.max())
            return fraction_of_positives, mean_predicted_value, lrn_score, prob_pos


        for lrn_clf, name in [(lrn, lrn_name)
                        # (isotonic, lrn_name + ' + Isotonic'),
                        # (sigmoid, lrn_name + ' + Sigmoid')
                        ]:
            fop, mpv, lss, pp = generate_calibration_data(lrn_clf,X_train,y_train,X_test,y_test)
            # ax1.plot(mpv, fop,alpha=0.75,label=f"{name} {lss:1.3f}")
            ax1.plot(mpv, fop,label=f"{name}")
            # ax2.hist(pp, range=(0, 1),linestyle="-", bins=10,\
            #  label=name,histtype="step",alpha=0.75, lw=2)

    ax1.set_ylabel("Fraction of positives")
    ax1.set_xlabel("Mean predicted value")
    ax1.set_ylim([-0.05, 1.05])
    ax1.legend(loc="lower right")

    # ax1.set_title('Calibration plots  (reliability curve)')

    # ax2.set_xlabel("Mean predicted value")
    # ax2.set_ylabel("Count")
    # ax2.legend(loc="upper center", ncol=2)
    # mpl.style.use("seaborn-paper")
    fig.tight_layout()
    # return fig, (ax1,ax2)
    return fig, (ax1)




 Prediction analysis

In [0]:
def perform_prediction(clf,clf_names, train_data,predict_data,bins=None,bins_labels=None,XY=None,export_prediction=False,export_csv=False):
    prediction = []
    prediction_classified = []
    freq_summary = []
    for lrn_clf, lrn_clf_name in zip(clf,clf_names) :
        # print(f"Train & and validate using the Following Parameters:")  
        lrn_clf_prediction = lrn_clf.fit(train_data[0], train_data[1]).predict_proba(predict_data)[:,-1]
        prediction.append(lrn_clf_prediction)

        if bins is not None:
            bins = np.array(bins)
            lrn_clf_prediction_classified = np.digitize(lrn_clf_prediction, bins)
            prediction_classified.append(lrn_clf_prediction_classified)

            unique_class, unique_count = np.unique(lrn_clf_prediction_classified,return_counts=True,axis=0)
            model = np.repeat(lrn_clf_name,len(unique_class))
            unique_count_percentage = (unique_count / np.sum(unique_count,axis=0))*100
            freq_summary.append(np.column_stack((unique_class,unique_count, unique_count_percentage,model)))

    freq_summary = np.row_stack((freq_summary))
    freq_summary = pd.DataFrame(data=freq_summary,columns=["class","count","percentage","model"])
    freq_summary["class"] = freq_summary["class"].astype(str)
    freq_summary["count"] = freq_summary["count"].astype(int)
    freq_summary["percentage"] = freq_summary["percentage"].astype(float)
    freq_summary = freq_summary.round({"percentage":3})

    if XY is not None:
        if export_prediction is True:
            prediction = np.column_stack((prediction))
            prediction = np.column_stack((XY,prediction))
            col_names =["x","y"] + clf_names
            prediction = pd.DataFrame(data=prediction,columns=col_names)
            if export_csv is True:
                prediction.to_csv("./Output/Tables/prediction.csv",index=False)
        else: 
            prediction_classified = np.column_stack((prediction_classified))
            prediction_classified = np.column_stack((XY,prediction_classified))
            col_names =["x","y"] + clf_names
            prediction_classified = pd.DataFrame(data=prediction_classified,columns=col_names)
            if export_csv is True:
                prediction_classified.to_csv("./Output/Tables/prediction.csv",index=False)
            # return prediction_classified, freq_summary

    else:
        pass

    return prediction, prediction_classified, freq_summary


def prediction_to_raster(clf_names,vrt_file,layers,outputSRS=None,outputBounds=None,width=None,height=None,noData=None):

    for lrn_clf_name in clf_names:
        output_file = "./Output/Figures/" + lrn_clf_name + ".tif"
        gridopt = gdal.GridOptions(format='GTiff',
        zfield=lrn_clf_name,layers=layers,
                        noData=noData,
                        # outputType=gdal.GDT_Int16,
                        outputSRS=outputSRS,
                        outputBounds=outputBounds,
                        width=width,height=height,
                        algorithm='nearest')
        output = gdal.Grid(output_file,vrt_file,options=gridopt)
        # output.FlushCache()
        output = None

        wrap_output_file = "./Output/Figures/" + lrn_clf_name + "_rs.tif"
        wrapopt = gdal.WarpOptions(outputBoundsSRS=outputSRS
                                    # xRes=50,yRes=50,
                                    )
        output = gdal.Warp(wrap_output_file,output_file,
                options=wrapopt
                # outputType=gdal.GDT_Int16,
                # xRes=45, yRes=45
                )
        # output.FlushCache()
        output = None

        # Replace wrapped file by the old one
        Path(output_file).unlink()
        Path(wrap_output_file).rename(output_file)

def extract_rs_unique_values(sampling_shp, raster,raster_names):
    summary = []
    for raster, raster_name in zip(rasters_list, models_names_list):

        src_ds=gdal.Open(raster) 
        gt=src_ds.GetGeoTransform()
        rb=src_ds.GetRasterBand(1)

        ds=ogr.Open(sampling_shp)
        lyr=ds.GetLayer()
        summary_raster = []
        for feat in lyr:
            geom = feat.GetGeometryRef()
            mx,my=geom.GetX(), geom.GetY()  #coord in map units

            #Convert from map to pixel coordinates.
            #Only works for geotransforms with no rotation.
            px = int((mx - gt[0]) / gt[1]) #x pixel
            py = int((my - gt[3]) / gt[5]) #y pixel

            intval=rb.ReadAsArray(px,py,1,1)
            if intval is None:
                intval = np.array([[4]])
            # intval=[4 if i is None else i for i in rb.ReadAsArray(px,py,1,1)]
            # print(intval[0]) #intval is a numpy array, length=1 as we only asked for 1 pixel value
            summary_raster.append(intval[0])

        summary_raster = np.row_stack((summary_raster))
        unique_class, unique_count = np.unique(summary_raster,return_counts=True,axis=0)
        unique_count_percentage = (unique_count / np.sum(unique_count,axis=0)) * 100
        model = np.repeat(raster_name,len(unique_class))
        summary.append(np.column_stack((unique_class,unique_count, unique_count_percentage,model)))

    summary = np.row_stack((summary))
    summary = pd.DataFrame(data=summary,columns=["class","count","percentage","model"])
    summary["class"] = summary["class"].astype(str)
    summary["count"] = summary["count"].astype(int)
    summary["percentage"] = summary["percentage"].astype(float)
    summary = summary.round({"percentage":3})

    return summary



In [0]:
# Init. Misc settings
# Setup the desired resampling Stratigy With 10 CV as Nested Inner Sampling
X_Data = input_df.drop(["Landslides"],axis=1,inplace=False).values
Y_Data = input_df["Landslides"].values               
inner_cv = KFold(n_splits=5, shuffle=True, random_state=seed)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=seed)

# Split the Input data acooring the resampling strategy
X_train_out,Y_train_out,X_test_out,Y_test_out = cv_splitting(X_Data, Y_Data, outer_cv)

# Evaluation metrics
score_metrics = {'AUC': 'roc_auc', 'Accuracy': make_scorer(accuracy_score),
                "Kappa":make_scorer(cohen_kappa_score)
                # "Logloss":make_scorer(log_loss,greater_is_better=False)
                }



 ## Tunning Random Forest

In [0]:
# # Init. model function for the objective function:
rf_clf = RandomForestClassifier()

# # Setting up number of evaluations and Trials object
n_evals = 30
rf_trails = Trials()


rf_search_space = {
        'max_features': "auto",
        'n_estimators': hp_scope.int(hp.quniform('n_estimators', 32,512,1)),
        'criterion': hp.choice('criterion', ["gini", "entropy"])
        }

# # Running optimization
rf_tune = fmin(partial(objective_func, clf= rf_clf, xdata=X_train_out, ydata=Y_train_out,cv_resample=inner_cv),
                space=rf_search_space, algo=tpe.suggest,
                trials=rf_trails, max_evals=n_evals, rstate=rng)

# Output the optimization history
rf_optimization_df, rf_best_hp = optimization_trails(rf_trails,rf_search_space)

# Plot optimization history of Random Forest
fig, ax = optimization_history(rf_trails)
plt.show()

# Generate final scores
rf_scores, rf_scores_raw  = tunned_model_evaluation(clf=rf_clf,opt_hp=space_eval(rf_search_space,rf_tune),x=X_Data,y=Y_Data,cv=outer_cv,scores=score_metrics)

# Generate and Compute ROC curve
_, rf_roc_data = generate_roc_data(rf_clf.set_params(**space_eval(rf_search_space,rf_tune)),X_train_out,Y_train_out,X_test_out,Y_test_out)



 ## Tunning Extra Trees

In [0]:
# # Init. model function for the objective function:
ext_clf = ExtraTreesClassifier()

# # Setting up number of evaluations and Trials object
n_evals = 30
ext_trails = Trials()

ext_search_space = {
        'max_features': "auto",
        'n_estimators': hp_scope.int(hp.quniform('n_estimators', 32,512,1)),
        'criterion': hp.choice('criterion', ["gini", "entropy"])
        }

# # Running optimization
ext_tune = fmin(partial(objective_func, clf= ext_clf, xdata=X_train_out, ydata=Y_train_out,cv_resample=inner_cv),
                space=ext_search_space, algo=tpe.suggest,
                trials=ext_trails, max_evals=n_evals, rstate=rng)

# Output the optimization history
ext_optimization_df, ext_best_hp = optimization_trails(ext_trails,ext_search_space)

# Plot optimization history of Random Forest
fig, ax = optimization_history(ext_trails)
plt.show()

# Generate final scores
ext_scores, ext_scores_raw  = tunned_model_evaluation(clf=ext_clf,opt_hp=space_eval(ext_search_space,ext_tune),x=X_Data,y=Y_Data,cv=outer_cv,scores=score_metrics)

# Generate and Compute ROC curve
_, ext_roc_data = generate_roc_data(ext_clf.set_params(**space_eval(ext_search_space,ext_tune)),X_train_out,Y_train_out,X_test_out,Y_test_out)



 ## Tunning Passive Aggressive Algorithms

In [0]:
# # Init. model function for the objective function:
pa_clf = PassiveAggressiveClassifier()

# # Setting up number of evaluations and Trials object
n_evals = 30
pa_trails = Trials()

pa_search_space = {
        'C': hp.loguniform('C', -3,3),
        'early_stopping': True,
        'n_iter_no_change': 10,
        'max_iter': hp_scope.int(hp.quniform('max_iter', 128,1024,1)),
        'fit_intercept': hp.choice('fit_intercept', [0, 1])
        }

# # Running optimization
pa_tune = fmin(partial(objective_func, clf= pa_clf, xdata=X_train_out, ydata=Y_train_out,cv_resample=inner_cv),
                space=pa_search_space, algo=tpe.suggest,
                trials=pa_trails, max_evals=n_evals, rstate=rng)

# Output the optimization history
pa_optimization_df, pa_best_hp = optimization_trails(pa_trails,pa_search_space)

# Plot optimization history of Random Forest
fig, ax = optimization_history(pa_trails)
plt.show()

# Generate final scores
pa_scores, pa_scores_raw  = tunned_model_evaluation(clf=pa_clf,opt_hp=space_eval(pa_search_space,pa_tune),x=X_Data,y=Y_Data,cv=outer_cv,scores=score_metrics)

# Generate and Compute ROC curve
_, pa_roc_data = generate_roc_data(pa_clf.set_params(**space_eval(pa_search_space,pa_tune)),X_train_out,Y_train_out,X_test_out,Y_test_out)



 ## Tunning Support Vector Machine

In [0]:
# Init. model function for the objective function:
svm_clf = SVC(probability=True)

# Setting up number of evaluations and Trials object
n_evals = 30
svm_trails = Trials()

svm_search_space =  hp.choice('kernel',
                                      ({'kernel': 'linear', 'C': hp.loguniform('linear_C', -8, 8)}, 
                                       {'kernel': 'rbf', 'C': hp.loguniform('rbf_C', -8, 8),"gamma": hp.loguniform('gamma', -8, 8)}))
ho_sample(svm_search_space)
# Running optimization
svm_tune = fmin(partial(objective_func, clf= svm_clf, xdata=X_train_out, ydata=Y_train_out,cv_resample=inner_cv),
                space=svm_search_space, algo=tpe.suggest,
                trials=svm_trails, max_evals=n_evals, rstate=rng)

# # Output the optimization history
svm_optimization_df, svm_best_hp = optimization_trails(svm_trails,svm_search_space)

# # Plot optimization history of Random Forest
fig, ax = optimization_history(svm_trails)
plt.show()

# # Generate final scores
svm_scores, svm_scores_raw  = tunned_model_evaluation(clf=svm_clf,opt_hp=space_eval(svm_search_space,svm_tune),x=X_Data,y=Y_Data,cv=outer_cv,scores=score_metrics)

# # Generate and Compute ROC curve
_, svm_roc_data = generate_roc_data(svm_clf.set_params(**space_eval(svm_search_space,svm_tune)),X_train_out,Y_train_out,X_test_out,Y_test_out)



 ## Tunning Gaussian Naive Bayes

In [0]:
# Init. model function for the objective function:
# nb_clf = BernoulliNB()
nb_clf = GaussianNB()

# Setting up number of evaluations and Trials object
n_evals = 2
nb_trails = Trials()

nb_search_space =  {
        # 'alpha': hp.uniform('alpha', 0.0, 2.0)
        "var_smoothing": 1e-9
        }

# Running optimization
nb_tune = fmin(partial(objective_func, clf= nb_clf, xdata=X_train_out, ydata=Y_train_out,cv_resample=inner_cv),
                space=nb_search_space, algo=tpe.suggest,
                trials=nb_trails, max_evals=n_evals, rstate=rng)

# Output the optimization history
nb_optimization_df, nb_best_hp = optimization_trails(nb_trails,nb_search_space)

# Plot optimization history of Random Forest
fig, ax = optimization_history(nb_trails)
plt.show()

# Generate final scores
nb_scores, nb_scores_raw = tunned_model_evaluation(clf=nb_clf,opt_hp=space_eval(nb_search_space,nb_tune),x=X_Data,y=Y_Data,cv=outer_cv,scores=score_metrics)

# Generate and Compute ROC curve
_, nb_roc_data = generate_roc_data(nb_clf.set_params(**space_eval(nb_search_space,nb_tune)),X_train_out,Y_train_out,X_test_out,Y_test_out)



 ## Tunning Quadratic Analysis

In [0]:
# Init. model function for the objective function:
qd_clf = QuadraticDiscriminantAnalysis()

# Setting up number of evaluations and Trials object
n_evals = 2
qd_trails = Trials()

qd_search_space =  {"tol": 1e-4}

# Running optimization
qd_tune = fmin(partial(objective_func, clf= qd_clf, xdata=X_train_out, ydata=Y_train_out,cv_resample=inner_cv),
                space=qd_search_space, algo=tpe.suggest,
                trials=qd_trails, max_evals=n_evals, rstate=rng)

# Output the optimization history
qd_optimization_df, qd_best_hp = optimization_trails(qd_trails,qd_search_space)

# Plot optimization history of Random Forest
fig, ax = optimization_history(qd_trails)
plt.show()

# Generate final scores
qd_scores, qd_scores_raw = tunned_model_evaluation(clf=qd_clf,opt_hp=space_eval(qd_search_space,qd_tune),x=X_Data,y=Y_Data,cv=outer_cv,scores=score_metrics)

# Generate and Compute ROC curve
_, qd_roc_data = generate_roc_data(qd_clf.set_params(**space_eval(qd_search_space,qd_tune)),X_train_out,Y_train_out,X_test_out,Y_test_out)


 ## Tunning Logistic Regression

In [0]:
# Init. model function for the objective function:
lr_clf = LogisticRegression()

# Setting up number of evaluations and Trials object
n_evals = 2
lr_trails = Trials()

lr_search_space =  {"C": 1.0,"solver":'lbfgs'}

# Running optimization
lr_tune = fmin(partial(objective_func, clf= lr_clf, xdata=X_train_out, ydata=Y_train_out,cv_resample=inner_cv),
                space=lr_search_space, algo=tpe.suggest,
                trials=lr_trails, max_evals=n_evals, rstate=rng)

# Output the optimization history
lr_optimization_df, lr_best_hp = optimization_trails(lr_trails,lr_search_space)

# Plot optimization history of Random Forest
fig, ax = optimization_history(lr_trails)
plt.show()

# Generate final scores
lr_scores, lr_scores_raw = tunned_model_evaluation(clf=lr_clf,opt_hp=space_eval(lr_search_space,lr_tune),x=X_Data,y=Y_Data,cv=outer_cv,scores=score_metrics)

# Generate and Compute ROC curve
_, lr_roc_data = generate_roc_data(lr_clf.set_params(**space_eval(lr_search_space,lr_tune)),X_train_out,Y_train_out,X_test_out,Y_test_out)


 ## Tunning K-Nearest Neighbors

In [0]:
# Init. model function for the objective function:
knn_clf = KNeighborsClassifier()

# Setting up number of evaluations and Trials object
n_evals = 30
knn_trails = Trials()

knn_search_space =  {
        'n_neighbors': hp_scope.int(hp.quniform('knn_n_neighbors', 1, 50,1)),
        'p': hp_scope.int(hp.quniform('p', 1, 5,1)),
        'weights': hp.choice('weights', ["uniform","distance"])
        }

# Running optimization
knn_tune = fmin(partial(objective_func, clf= knn_clf, xdata=X_train_out, ydata=Y_train_out,cv_resample=inner_cv),
                space=knn_search_space, algo=tpe.suggest,
                trials=knn_trails, max_evals=n_evals, rstate=rng)

# Output the optimization history
knn_optimization_df, knn_best_hp = optimization_trails(knn_trails,knn_search_space)

# Plot optimization history of Random Forest
fig, ax = optimization_history(knn_trails)
plt.show()

# Generate final scores
knn_scores, knn_scores_raw = tunned_model_evaluation(clf=knn_clf,opt_hp=space_eval(knn_search_space,knn_tune),x=X_Data,y=Y_Data,cv=outer_cv,scores=score_metrics)

# Generate and Compute ROC curve
_, knn_roc_data = generate_roc_data(knn_clf.set_params(**space_eval(knn_search_space,knn_tune)),X_train_out,Y_train_out,X_test_out,Y_test_out)



 ## Tunning Decision Trees

In [0]:
# Init. model function for the objective function:
dt_clf = DecisionTreeClassifier()

# Setting up number of evaluations and Trials object
n_evals = 30
dt_trails = Trials()

dt_search_space = {
        'min_weight_fraction_leaf': hp.uniform('min_weight_fraction_leaf', 0,0.5),
        'max_features': hp.uniform('max_features', 0,1),
        'criterion': hp.choice('criterion', ["gini", "entropy"])
        }

# Running optimization
dt_tune = fmin(partial(objective_func, clf= dt_clf, xdata=X_train_out, ydata=Y_train_out,cv_resample=inner_cv),
                space=dt_search_space, algo=tpe.suggest,
                trials=dt_trails, max_evals=n_evals, rstate=rng)

# Output the optimization history
dt_optimization_df, dt_best_hp = optimization_trails(dt_trails,dt_search_space)

# Plot optimization history of Random Forest
fig, ax = optimization_history(dt_trails)
plt.show()

# Generate final scores
dt_scores, dt_scores_raw = tunned_model_evaluation(clf=DecisionTreeClassifier(),opt_hp=space_eval(dt_search_space,dt_tune),x=X_Data,y=Y_Data,cv=outer_cv,scores=score_metrics)

# Generate and Compute ROC curve
_, dt_roc_data = generate_roc_data(DecisionTreeClassifier(**space_eval(dt_search_space,dt_tune)),X_train_out,Y_train_out,X_test_out,Y_test_out)


 ## Tunning Neural Network

In [0]:
# Init. model function for the objective function:
nnet_clf = MLPClassifier()

# Setting up number of evaluations and Trials object
n_evals = 30
nnet_trails = Trials()

nnet_search_space = {
        'hidden_layer_sizes': hp_scope.int(hp.quniform('hidden_layer_sizes', 3,54,1)),
        'activation': 'relu',
        'solver': 'lbfgs',
        'max_iter': 128
        }

# Running optimization
nnet_tune = fmin(partial(objective_func, clf= nnet_clf, xdata=X_train_out, ydata=Y_train_out,cv_resample=inner_cv),
                space=nnet_search_space, algo=tpe.suggest,
                trials=nnet_trails, max_evals=n_evals, rstate=rng)

# Output the optimization history
nnet_optimization_df, nnet_best_hp = optimization_trails(nnet_trails,nnet_search_space)

# Plot optimization history of Random Forest
fig, ax = optimization_history(nnet_trails)
plt.show()

# Generate final scores
nnet_scores, nnet_scores_raw = tunned_model_evaluation(clf=MLPClassifier(),opt_hp=space_eval(nnet_search_space,nnet_tune),x=X_Data,y=Y_Data,cv=outer_cv,scores=score_metrics)

# Generate and Compute ROC curve
_, nnet_roc_data = generate_roc_data(MLPClassifier(**space_eval(nnet_search_space,nnet_tune)),X_train_out,Y_train_out,X_test_out,Y_test_out)


 ## Tunning LightGBM

In [0]:
# # Init. model function for the objective function:
lgb_clf = LGBMClassifier()

# # Setting up number of evaluations and Trials object
n_evals = 30
lgb_trails = Trials()

lgb_search_space = {
        # "max_depth" : hp_scope.int(hp.quniform("max_depth", 2,20, 1)),
        "learning_rate" : hp.uniform("learning_rate", 1e-3, 1),
        #"learning_rate" : hp.uniform("learning_columnsrate", 1e-3, 1),
        # "subsample" : hp.uniform("subsample", 0.5, 1),
        "num_leaves" : hp_scope.int(hp.quniform("num_leaves",4,1024,1)), # < 2**depth for 0.6  
        "n_estimators" : hp_scope.int(hp.quniform("n_estimators",16,1024,1))
        }

# # Running optimization
lgb_tune = fmin(partial(objective_func, clf= lgb_clf, xdata=X_train_out, ydata=Y_train_out,cv_resample=inner_cv),
                space=lgb_search_space, algo=tpe.suggest,
                trials=lgb_trails, max_evals=n_evals, rstate=rng)

# Output the optimization history
lgb_optimization_df, lgb_best_hp = optimization_trails(lgb_trails,lgb_search_space)

# Plot optimization history of Random Forest
fig, ax = optimization_history(lgb_trails)
plt.show()

# Generate final scores
lgb_scores, lgb_scores_raw  = tunned_model_evaluation(clf=lgb_clf,opt_hp=space_eval(lgb_search_space,lgb_tune),x=X_Data,y=Y_Data,cv=outer_cv,scores=score_metrics)

# Generate and Compute ROC curve
_, lgb_roc_data = generate_roc_data(lgb_clf.set_params(**space_eval(lgb_search_space,lgb_tune)),X_train_out,Y_train_out,X_test_out,Y_test_out)


 ## Tunning Bart

In [0]:
# # Init. model function for the objective function:
from bartpy.sklearnmodel import SklearnModel

bart_clf = SklearnModel()

# # Setting up number of evaluations and Trials object
n_evals = 10
bart_trails = Trials()

bart_search_space = {
        "n_trees" : hp_scope.int(hp.quniform("n_trees", 16,256, 1)),
        # "p_grow" : hp.uniform("p_grow", 0.5, 1),
        "n_samples" : hp_scope.int(hp.quniform("n_samples",16,256,1)), # < 2**depth for 0.6  
        "n_burn" : hp_scope.int(hp.quniform("n_burn",16,256,1))
        }

# # Running optimization
bart_tune = fmin(partial(objective_func, clf= bart_clf, xdata=X_train_out, ydata=Y_train_out,cv_resample=inner_cv),
                space=bart_search_space, algo=tpe.suggest,
                trials=bart_trails, max_evals=n_evals, rstate=rng)

# Output the optimization history
bart_optimization_df, bart_best_hp = optimization_trails(bart_trails,bart_search_space)

# Plot optimization history of Random Forest
fig, ax = optimization_history(bart_trails)
plt.show()

# Generate final scores
bart_scores, bart_scores_raw  = tunned_model_evaluation(clf=bart_clf,opt_hp=space_eval(bart_search_space,bart_tune),x=X_Data,y=Y_Data,cv=outer_cv,scores=score_metrics)

# Generate and Compute ROC curve
_, bart_roc_data = generate_roc_data(bart_clf.set_params(**space_eval(bart_search_space,bart_tune)),X_train_out,Y_train_out,X_test_out,Y_test_out)


In [0]:
models_names_list = ['DT','EXT','KNN','LGB','LR','NB','NNET','QDA','RF','SVM']
models_raw_scores_list = [
                          dt_scores_raw["test_AUC"],
                          ext_scores_raw["test_AUC"],
                          knn_scores_raw["test_AUC"],
                          lgb_scores_raw["test_AUC"],
                          lr_scores_raw["test_AUC"],
                          nb_scores_raw["test_AUC"],
                          nnet_scores_raw["test_AUC"],
                          qd_scores_raw["test_AUC"],
                          rf_scores_raw["test_AUC"],
                          svm_scores_raw["test_AUC"]
                          ]

sig_df = sig_stat(models_raw_scores_list,models_names_list)
with open("./Output/Tables/sig_test.md",'w') as tabl:
    print(tabulate(sig_df, headers=['Pairwise', 'p.value','t.value'],tablefmt="grid"),file=tabl)



In [0]:
models_roc_data_list = [
                          dt_roc_data,
                          ext_roc_data,
                          knn_roc_data,
                          lgb_roc_data,
                          lr_roc_data,
                          nb_roc_data,
                          nnet_roc_data,
                          qd_roc_data,
                          rf_roc_data,
                          svm_roc_data
                          ]

fig, ax = plot_roc_curves(models_roc_data_list,models_names_list)
ax.set_xlim(0, 1.01)
ax.set_ylim(0, 1.01)

plt.legend(loc="best")
fig.tight_layout()
plt.show()
fig.savefig("./Output/Figures/ROC_plot1.jpg",dpi=600, quality=100,bbox_inchs="tight")


In [0]:
models_list = [
               dt_clf.set_params(**space_eval(dt_search_space,dt_tune)),
               ext_clf.set_params(**space_eval(ext_search_space,ext_tune)),
               knn_clf.set_params(**space_eval(knn_search_space,knn_tune)),
               lgb_clf.set_params(**space_eval(lgb_search_space,lgb_tune)),
               lr_clf.set_params(**space_eval(lr_search_space,lr_tune)),
               nb_clf.set_params(**space_eval(nb_search_space,nb_tune)),
               nnet_clf.set_params(**space_eval(nnet_search_space,nnet_tune)),
               qd_clf.set_params(**space_eval(qd_search_space,qd_tune)),
               rf_clf.set_params(**space_eval(rf_search_space,rf_tune)),
               svm_clf.set_params(**space_eval(svm_search_space,svm_tune))
               ]

fig, ax = plot_calibration_curves(clf=models_list,x=X_Data ,y=Y_Data, test_size=0.3, cv = 10, seed=seed, clf_name=models_names_list)
ax.set_xlim(0, 1.01)
ax.set_ylim(0, 1.01)

plt.legend(loc="best")
fig.tight_layout()
plt.show()
fig.savefig("./Output/Figures/Calibration_plot1.jpg",dpi=600, quality=100,bbox_inchs="tight")



In [0]:
st_data_grid = pd.read_hdf('./Data/Predict/Study_Area45_Array.hdf').iloc[:,0:16]
st_xy_grid = pd.read_hdf('./Data/Predict/Study_Area45_Array.hdf').values[:,-2:]
bins= [0.05,0.30,0.50,0.75,np.inf]
bins_labels=["Very Low", "Low", "Moderate", "High", "Very High"]
bins_labels_replace = {"0":"Very Low", "1":"Low", "2":"Moderate", "3":"High", "4":"Very High",
                        "0.0":"Very Low", "1.0":"Low", "2.0":"Moderate", "3.0":"High", "4.0":"Very High"}
_, prediction_data, prediction_summary = perform_prediction(clf=models_list,clf_names=models_names_list,\
train_data=[X_Data, Y_Data],predict_data=st_data_grid,\
bins=bins,bins_labels=bins_labels,XY=st_xy_grid,export_prediction=False,export_csv=True)
prediction_summary = prediction_summary.replace({"class":bins_labels_replace})

vrt_file = "./Output/Tables/prediction.vrt"
outputBounds,width, height = [305739,4054353,224105,4007733], 1632.68, 932.4
prediction_to_raster(clf_names=models_names_list,vrt_file=vrt_file,
                    layers="prediction",outputSRS="EPSG:32632",
                    outputBounds=outputBounds,width=width,height=height,noData=-9999)
sample_shp = "./Data/shp/Ls_Pts.shp"
rasters_list = ["./Output/Figures/" + model + ".tif" for model in models_names_list]
prediction_overly_summary = extract_rs_unique_values(sample_shp, rasters_list,models_names_list)
prediction_overly_summary = prediction_overly_summary.replace({"class":bins_labels_replace})




In [0]:
df = prediction_summary.pivot("model","class","percentage")
df = df.reindex(["Very Low","Low","Moderate","High","Very High"],axis=1)
df.to_csv("./Output/Tables/prediction_summary_pivot.csv",index=True,header=True)
df = prediction_summary.pivot("model","class","percentage")
prediction_summary.to_csv("./Output/Tables/prediction_summary.csv",index=None,header=True)

fig, ax1 = plt.subplots(figsize=(7,7))
bar_width = 0.375
y_pos = np.arange(0,2*len(df),2)
yticks = len(df.columns)*bar_width/2 + y_pos

for i, col in enumerate(df.columns):
    y_pos + i * bar_width
    bars = ax1.barh(y_pos + i * bar_width, df[col],
    height = bar_width,
    label = col)
    for bar in bars:
        # print(bar.get_width())
        ax1.annotate(f"{round(bar.get_width(),2)}%",
                xy=(bar.get_width() + 5,
                    bar.get_y() + bar.get_height()/2
                ),
                ha="center", va='center',
                color=bar.get_facecolor(),size=10,fontweight="medium")
ax1.legend(loc="upper right")
# Add some text for labels, title and custom x-axis tick labels, etc.
ax1.set_xlabel("Area Extent (%)")
ax1.set_ylabel("")
ax1.set_yticks(yticks)
ax1.set_yticklabels(df.index)
ax1.set_ylim([0 - bar_width,20])
ax1.set_xlim([-0.05,50.5])
ax1.xaxis.grid(False)
ax1.yaxis.grid(False)
# Save the figure and show
fig.tight_layout()
plt.tight_layout()
fig.savefig("./Output/Figures/area_extent.jpg",dpi=600, quality=100,bbox_inchs="tight")





In [0]:

df = prediction_overly_summary.pivot("model","class","percentage")
df = df.reindex(["Very Low","Low","Moderate","High","Very High"],axis=1)
df.to_csv("./Output/Tables/prediction_overly_summary_pivot.csv",index=True,header=True)
prediction_overly_summary.to_csv("./Output/Tables/prediction_overly_summary.csv",index=None,header=True)

fig, ax1 = plt.subplots(figsize=(7,7))
bar_width = 0.375
y_pos = np.arange(0,2*len(df),2)
yticks = len(df.columns)*bar_width/2 + y_pos

for i, col in enumerate(df.columns):
    y_pos + i * bar_width
    bars = ax1.barh(y_pos + i * bar_width, df[col],
    height = bar_width,
    label = col)
    for bar in bars:
        # print(bar.get_width())
        ax1.annotate(f"{round(bar.get_width(),2)}%",
                xy=(bar.get_width() + 5,
                    bar.get_y() + bar.get_height()/2
                ),
                ha="center", va='center',
                color=bar.get_facecolor(),size=10,fontweight="medium")
ax1.legend(loc="upper right")
# Add some text for labels, title and custom x-axis tick labels, etc.
ax1.set_xlabel("Landslide Distribution (%)")
ax1.set_ylabel("")
ax1.set_yticks(yticks)
ax1.set_yticklabels(df.index)
ax1.set_ylim(0-bar_width,20)
ax1.set_xlim([-0.05,102.5])
ax1.xaxis.grid(False)
ax1.yaxis.grid(False)
# Save the figure and show
fig.tight_layout()
plt.tight_layout()
fig.savefig("./Output/Figures/ls_distr.jpg",dpi=600, quality=100,bbox_inchs="tight")


In [0]:
# Output the Optimium parameters as md table
hyper_opt = pd.concat([rf_best_hp.T,svm_best_hp.T,nb_best_hp.T,knn_best_hp.T,dt_best_hp.T,nnet_best_hp.T,lgb_best_hp.T])
hyper_opt = pd.concat([
                        #   bart_best_hp.T,
                          dt_best_hp.T,
                          ext_best_hp.T,
                          knn_best_hp.T,
                          lgb_best_hp.T,
                          lr_best_hp.T,
                          nb_best_hp.T,
                          nnet_best_hp.T,
                        #   pa_best_hp.T,
                          qd_best_hp.T,
                          rf_best_hp.T,
                          svm_best_hp.T
                          ])

hyper_opt = hyper_opt.rename(columns={0: "optimium","0": "optimium"})
hyper_opt["hyperparamter"] = hyper_opt.index
hyper_opt = hyper_opt.round({"optimium":4})
with open("./Output/Tables/hyper_opt.md",'w') as tabl:
    print(tabulate(hyper_opt, headers=['Hyperparamter', 'Optimium'],tablefmt="grid"),file=tabl)

In [0]:
# Output the scores results as md table
score_opt = pd.concat([
dt_scores.loc[["test_AUC","test_Accuracy","test_Kappa"],["mean","std"]].T,
ext_scores.loc[["test_AUC","test_Accuracy","test_Kappa"],["mean","std"]].T,
knn_scores.loc[["test_AUC","test_Accuracy","test_Kappa"],["mean","std"]].T,
lgb_scores.loc[["test_AUC","test_Accuracy","test_Kappa"],["mean","std"]].T,
lr_scores.loc[["test_AUC","test_Accuracy","test_Kappa"],["mean","std"]].T,
nb_scores.loc[["test_AUC","test_Accuracy","test_Kappa"],["mean","std"]].T,
nnet_scores.loc[["test_AUC","test_Accuracy","test_Kappa"],["mean","std"]].T,
qd_scores.loc[["test_AUC","test_Accuracy","test_Kappa"],["mean","std"]].T,
rf_scores.loc[["test_AUC","test_Accuracy","test_Kappa"],["mean","std"]].T,
svm_scores.loc[["test_AUC","test_Accuracy","test_Kappa"],["mean","std"]].T
])
score_opt = score_opt.round({"test_AUC":3,"test_Accuracy":3,"test_Kappa":3})
with open("./Output/Tables/score_opt.md",'w') as tabl:
    print(tabulate(score_opt, headers=['AUC', 'ACC','Kappa'],tablefmt="grid"),file=tabl)

# Build the plot
# Define labels, positions, bar heights and error bar heights
for idx, metric in enumerate(["AUC","ACC","Kappa"]):
    # models_names_list = ["RF","SVM","NB","KNN","DT","NNET","LGB"]
    y_pos = np.arange(len(models_names_list))
    ct = [score_opt.iloc[i,idx] for i in range(0,len(score_opt),2)]
    error = [score_opt.iloc[i,idx] for i in range(1,len(score_opt),2)]
    plt.style.use("seaborn-paper")
    fig, ax = plt.subplots(figsize=(5,5))
    ax.barh(y_pos,ct,
        xerr=error,
        align='center',
        alpha=0.5,
        ecolor='black',
        capsize=10)
    ax.set_xlabel(metric)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(models_names_list)
    # ax.set_title('Coefficent of Thermal Expansion (CTE) of Three Metals')
    ax.yaxis.grid(False)
    ax.xaxis.grid(False)

    # Save the figure and show
    fig.tight_layout()
    plt.tight_layout()
    plt.savefig('./Output/Figures/'+metric+'.jpg',quality=100,dpi=300,bbox_inchs="tight")
    plt.show()

